# Napari image viewer

Uses the octopuslite dask image loader to automatically sort and load images from multichannel time lapse experiments. Can also use btrack to load associated tracks and segmentation images.

In [1]:
import napari
from octopuslite import DaskOctopusLiteLoader

ModuleNotFoundError: No module named 'octopuslite'

### Simple view 

In [44]:
images = DaskOctopusLiteLoader('/run/user/1000/gvfs/smb-share:server=lowe-sn00.biochem.ucl.ac.uk,share=lowegrp/Data/Nathan/kraken/ras/ND0006/Pos8', crop=(1200,1600))

Using cropping: (1200, 1600)


# Cropping for sake of tests

In [45]:
gfp = images['gfp'][0:300,:,:]
rfp = images['rfp'][0:300,:,:]
irfp = images['irfp'][0:300,:,:]

In [46]:
viewer = napari.Viewer()
#viewer.add_image(stack[0], name = "brightfield", contrast_limits=[0,200], blending = 'additive')#, is_pyramid=False)
viewer.add_image(gfp, name="gfp", 
                 #contrast_limits=[100,255],
                 blending = 'additive', colormap = 'green')#, is_pyramid=False)
viewer.add_image(rfp, name="rfp", 
                 #contrast_limits=[0,30], 
                 blending = 'additive', colormap = 'magenta')#, is_pyramid=False)
viewer.add_image(irfp, name="irfp", 
                 contrast_limits=[0,255], 
                 blending = 'additive', colormap = 'cyan')#, is_pyramid=False)
#viewer.add_image(seg['mask'], name = 'masks')
#viewer.add_tracks(gfp_tracks)

<Image layer 'irfp' at 0x7f6e39d4cf70>

In [31]:
viewer = napari.Viewer()
#viewer.add_image(stack[0], name = "brightfield", contrast_limits=[0,200], blending = 'additive')#, is_pyramid=False)
viewer.add_image(images['gfp'], name="gfp", 
                 #contrast_limits=[100,255],
                 blending = 'additive', colormap = 'green')#, is_pyramid=False)
viewer.add_image(images['rfp'], name="rfp", 
                 #contrast_limits=[0,30], 
                 blending = 'additive', colormap = 'magenta')#, is_pyramid=False)
viewer.add_image(images['irfp'], name="irfp", 
                 #contrast_limits=[0,255], 
                 blending = 'additive', colormap = 'cyan')#, is_pyramid=False)
#viewer.add_image(seg['mask'], name = 'masks')
#viewer.add_tracks(gfp_tracks)

<Image layer 'irfp' at 0x7f6e70298a60>

### Add tracks

In [3]:
seg = DaskOctopusLiteLoader('/home/nathan/data/kraken/test/MK0003/Pos15/Pos15_stardist_masks')

Using cropping: None


In [4]:
import btrack
from btrack.utils import tracks_to_napari

In [8]:
with btrack.dataio.HDF5FileHandler('/home/nathan/data/kraken/test/MK0003/Pos15/tracking.h5', 'r', obj_type='obj_type_1') as h:
    gfp_tracks = h.tracks

[INFO][2021/11/18 03:11:53 pm] Opening HDF file: /home/nathan/data/kraken/test/MK0003/Pos15/tracking.h5...
[INFO][2021/11/18 03:11:53 pm] Loading tracks/obj_type_1
[INFO][2021/11/18 03:11:54 pm] Loading objects/obj_type_1 (77222, 5) (77222 filtered: None)
[INFO][2021/11/18 03:11:56 pm] Closing HDF file: /home/nathan/data/kraken/test/MK0003/Pos15/tracking.h5


In [9]:
gfp_tracks, properties, graph = tracks_to_napari(gfp_tracks, ndim = 2)

### Concatenating sequential stacks

In [25]:
images1 = DaskOctopusLiteLoader('/run/user/1000/gvfs/smb-share:server=lowe-sn00.biochem.ucl.ac.uk,share=lowegrp/Data/Nathan/kraken/ras/ND0007/Pos0', crop=(1200,1600))

Using cropping: (1200, 1600)


In [14]:
images_2 = DaskOctopusLiteLoader('/run/user/1000/gvfs/smb-share:server=lowe-sn00.biochem.ucl.ac.uk,share=lowegrp/Data/Nathan/kraken/ras/ND0008/Pos0', crop=(1200,1600))

Using cropping: (1200, 1600)


In [26]:
import numpy as np
rfp = np.concatenate((images0['rfp'],images1['rfp'], images_2['rfp']), axis = 0)
gfp = np.concatenate((images0['gfp'],images1['gfp'], images_2['gfp']), axis = 0)
irfp = np.concatenate((images0['irfp'],images1['irfp'], images_2['irfp']), axis = 0)

In [27]:
viewer = napari.Viewer()
#viewer.add_image(stack[0], name = "brightfield", contrast_limits=[0,200], blending = 'additive')#, is_pyramid=False)
viewer.add_image(gfp, name="gfp", 
                 #contrast_limits=[100,255],
                 blending = 'additive', colormap = 'green')#, is_pyramid=False)
viewer.add_image(rfp, name="rfp", 
                 #contrast_limits=[0,30], 
                 blending = 'additive', colormap = 'magenta')#, is_pyramid=False)
viewer.add_image(irfp, name="irfp", 
                 contrast_limits=[0,255], 
                 blending = 'additive', colormap = 'cyan')#, is_pyramid=False)
#viewer.add_image(seg['mask'], name = 'masks')
#viewer.add_tracks(gfp_tracks)

<Image layer 'irfp' at 0x7f6e71c360d0>